# Running the Luminosity-Temperature-Radius pipeline

We will use the XGA LTR pipeline to try to determine overdensity radii, and the global temperature and luminosity measurements within them. This process is [fully described here](https://xga.readthedocs.io/en/latest/notebooks/pipeline_tutorials/LT_pipeline.html). This should be relatively successful, even given the low surface brightness of the groups, as it easier to get temperature constraints on lower-temperature systems than hotter systems when using XMM.

Visualisations will also be generated here, to demonstrate the regions that were determined to be contaminating sources and masked out, as some adjustment may be required given the nature of these groups.

## Import statements

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity
import os
from shutil import rmtree

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 40
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.tools import luminosity_temperature_pipeline
from xga.relations.clusters.RT import arnaud_r2500

%matplotlib inline

## Reading the sample file

Reading in the positions, names, and redshifts of the groups - the pipeline expects position columns to be called 'ra' and 'dec', so those are renamed:

In [2]:
samp = pd.read_csv('../sample_files/init_group_info.csv')
samp = samp.rename(columns={'est_ra': 'ra', 'est_dec': 'dec'})
samp

,name,ra,dec,redshift
0,25124,243.629055,26.730240,0.186
1,44739,229.793620,28.331750,0.118
2,19670A,150.256710,50.793942,0.134
3,19670B,150.214920,50.805014,0.134
4,12833,129.496800,44.248700,0.145
5,44858,230.112000,28.887750,0.127
6,PHL1811,328.769920,-9.588805,0.077


## Running the pipeline (twice)

The LTR will be run twice, targeting two different overdensity radii. The pipeline runs will save the results to disk (as we are passing paths for the results and radii history files), as well as returning the results and the XGA ClusterSample object generated by the procedure.

### $R_{500}$

The first overdensity radius we are targeting is $R_{500}$, which is the default for this pipeline. The Arnaud et al. (2005) $R_{500}$-$T_{\rm{X}}$ is used for the process, and the starting radius is set to 200 kpc. Metallicity, nH, and redshift are all frozen during the spectral fitting process.

In [3]:
res_file = '../outputs/results/ltr_r500_pipeline_results.csv'
rhist_file = '../outputs/results/radii_r500_history.csv'

srcs, results, rad_hist = luminosity_temperature_pipeline(samp, Quantity(200, 'kpc'), timeout=Quantity(4, 'hr'), 
                                                          save_samp_results_path=res_file,
                                                          save_rad_history_path=rhist_file)

Setting up Galaxy Clusters: 100%|██████████| 5/5 [00:02<00:00,  1.82it/s]
/mnt/ufs18/home-218/turne540/code/XGA/xga/tools/clusters/LT.py:166: UserWarning: The following do not appear to have any XMM data, and will not be included in the sample (can also check .failed_names); 25124, 44739
  samp = ClusterSample(sample_data['ra'].values, sample_data['dec'].values, sample_data['redshift'].values,
/mnt/ufs18/home-218/turne540/code/XGA/xga/samples/extended.py:237: UserWarning: Non-fatal warnings occurred during the declaration of some sources, to access them please use the suppressed_warnings property of this sample.
  self._check_source_warnings()
/mnt/ufs18/home-218/turne540/code/XGA/xga/xspec/run.py:186: UserWarning: All XSPEC operations had already been run.
  warnings.warn("All XSPEC operations had already been run.")
/mnt/ufs18/home-218/turne540/code/XGA/xga/products/relation.py:704: UserWarning: Some of the x values you have passed are outside the validity range of this relation (1.0

### $R_{2500}$

The other overdensity radius we are targeting is $R_{2500}$. The Arnaud et al. (2005) $R_{2500}$-$T_{\rm{X}}$ is used for the process. As the ClusterSample in this run is completely separate from the $R_{500}$ run, it is possible that slightly different regions have been selected as contaminating sources, this would be due to XGA ClusterSamples allowing point sources in the core region to not be masked out, in case they are misidentified cool-cores.

In [4]:
res_file = '../outputs/results/ltr_r2500_pipeline_results.csv'
rhist_file = '../outputs/results/radii_r2500_history.csv'

srcs_r2500, results_r2500, \
    rad_hist_r2500 = luminosity_temperature_pipeline(samp, Quantity(200, 'kpc'), timeout=Quantity(4, 'hr'), 
                                                     rad_temp_rel=arnaud_r2500, save_samp_results_path=res_file,
                                                     save_rad_history_path=rhist_file)

Setting up Galaxy Clusters: 100%|██████████| 5/5 [00:02<00:00,  1.80it/s]
/mnt/ufs18/home-218/turne540/code/XGA/xga/tools/clusters/LT.py:166: UserWarning: The following do not appear to have any XMM data, and will not be included in the sample (can also check .failed_names); 25124, 44739
  samp = ClusterSample(sample_data['ra'].values, sample_data['dec'].values, sample_data['redshift'].values,
/mnt/ufs18/home-218/turne540/code/XGA/xga/samples/extended.py:237: UserWarning: Non-fatal warnings occurred during the declaration of some sources, to access them please use the suppressed_warnings property of this sample.
  self._check_source_warnings()
/mnt/ufs18/home-218/turne540/code/XGA/xga/xspec/run.py:186: UserWarning: All XSPEC operations had already been run.
  warnings.warn("All XSPEC operations had already been run.")
/mnt/ufs18/home-218/turne540/code/XGA/xga/products/relation.py:704: UserWarning: Some of the x values you have passed are outside the validity range of this relation (1.0

In [7]:
results

,name,ra,dec,redshift,r500,Tx500,Tx500-,Tx500+,Lx500_0.5-2.0,Lx500_0.5-2.0-,Lx500_0.5-2.0+,Lx500_0.01-100.0,Lx500_0.01-100.0-,Lx500_0.01-100.0+
2,19670A,150.25671,50.793942,0.134,565.230745,1.69940,0.301139,0.625748,1.923919e+42,3.586698e+41,4.989918e+41,4.100454e+42,7.430867e+41,9.649734e+41
3,19670B,150.21492,50.805014,0.134,590.155347,1.92186,0.236760,0.412706,4.771068e+42,2.548252e+41,3.382819e+41,1.056385e+43,9.319743e+41,1.084395e+42
4,12833,129.49680,44.248700,0.145,501.780182,1.41992,0.092045,0.125536,3.899479e+42,3.388153e+41,2.897504e+41,7.836917e+42,7.118439e+41,7.602804e+41
5,44858,230.11200,28.887750,0.127,726.221353,2.65845,0.234188,0.275686,2.566063e+43,1.004189e+42,1.104279e+42,6.352387e+43,4.070900e+42,3.823138e+42
6,PHL1811,328.76992,-9.588805,0.077,563.093086,1.68634,0.058023,0.070412,7.743633e+42,3.934662e+41,2.703076e+41,1.746595e+43,1.121958e+42,8.685135e+41


In [6]:
results_r2500

,name,ra,dec,redshift,r2500,Tx2500,Tx2500-,Tx2500+,Lx2500_0.5-2.0,Lx2500_0.5-2.0-,Lx2500_0.5-2.0+,Lx2500_0.01-100.0,Lx2500_0.01-100.0-,Lx2500_0.01-100.0+
2,19670A,150.25671,50.793942,0.134,229.549742,1.44342,0.418233,0.794367,2.921256e+41,1.848456e+41,1.648231e+41,5.906510e+41,3.578155e+41,4.199029e+41
3,19670B,150.21492,50.805014,0.134,276.657453,3.23768,0.660389,0.875739,1.750286e+42,2.128867e+41,1.904279e+41,4.686904e+42,6.914031e+41,8.544892e+41
4,12833,129.49680,44.248700,0.145,226.670867,1.42136,0.091381,0.118683,2.250820e+42,1.516062e+41,2.161923e+41,4.525233e+42,4.464786e+41,4.542320e+41
5,44858,230.11200,28.887750,0.127,317.285494,2.58898,0.182149,0.199412,1.309734e+43,4.919890e+41,4.827326e+41,3.210352e+43,1.479570e+42,1.725091e+42
6,PHL1811,328.76992,-9.588805,0.077,257.628544,1.73804,0.067516,0.072468,5.564601e+42,3.559554e+41,2.093395e+41,1.265809e+43,7.805432e+41,4.518125e+41
